# Librerías

In [ ]:
# Tratamiento de datos
# ==============================================================================
import pandas as pd

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt

# Modelado y Forecasting
# ==============================================================================
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import  KNeighborsRegressor

from keras.models import Sequential
from keras.layers import Conv1D, Dense, Flatten

from statsmodels.tsa.statespace.sarimax import SARIMAX


# Calculo de metricas
# ==============================================================================
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Constantes

In [ ]:
FILE_DATOS = './data/lagoon_hourly_filled.csv'

HOR_31 = 120
HOR_32 = 400
HOR_33 = 420

HORIZONTES_3 = [HOR_31, HOR_32, HOR_33]

PRIMER_CONJUNTO = 1110
SEGUNDO_CONJUNTO = 1500
TERCER_CONJUNTO = 2113

# Carga de datos

In [ ]:
parametro = 'temperatura'
output=parametro
inputs = [
          'fecha', 'temperatura',
          'ambiente', 'nivel'
          ]
usecols = inputs.copy()

datos = pd.read_csv(FILE_DATOS,
                    sep=',', 
                    usecols = usecols)

# Preparación del dato
# ==============================================================================
datos['fecha'] = pd.to_datetime(datos['fecha'], format='%d/%m/%Y %H:%M')
datos = datos.set_index('fecha')
datos = datos.rename(columns={'x': 'y'})
datos = datos.asfreq('H')
datos = datos.sort_index()

datos.info()

datos.head()

# Modelos

In [ ]:
def separar_train_test(datos, conjunto, horizonte):
    data = datos[:conjunto].copy()
    steps = horizonte
    
    datos_train = data[:-steps]
    datos_test  = data[-steps:]

    print(f'Fechas train : {datos_train.index.min()} --- {datos_train.index.max()}  (n={len(datos_train)})')
    print(f'Fechas test  : {datos_test.index.min()} --- {datos_test.index.max()}  (n={len(datos_test)})')

    fig, ax = plt.subplots(figsize=(20, 8))
    datos_train['temperatura'].plot(ax=ax, label='entrenamiento')
    datos_test['temperatura'].plot(ax=ax, label='test')
    ax.set_title(f'Conjunto de entrenamiento y test - Dataset {conjunto} ({horizonte} horas)')
    ax.legend()
    plt.show()

    return datos_train, datos_test
    

In [ ]:
def obtener_metricas(test, predictions):
    r2 = r2_score(test, predictions)
    mse = mean_squared_error(test, predictions)
    mae = mean_absolute_error(test, predictions)

    return r2, mse, mae

In [ ]:
def pintar_grafica(titulo,data, label_datos, colores, label_x, label_y, alto, ancho):
    fig, ax = plt.subplots(figsize=(ancho, alto))

    ax.minorticks_on()

    # Graficar la serie de datos con pandas en el eje configurado
    for id in range(0,len(data)):
        data[id].plot(ax=ax, label=label_datos[id], color=colores[id],lw = 2)

    # Aplicar configuraciones al eje
    ax.set_facecolor('white')
    ax.grid(True, color='lightgrey', linestyle='--', linewidth=0.5)

    # Ajustes finales de estilo si son necesarios
    # ax.tick_params(labelsize=20)
    ax.tick_params(axis='x', which='major', labelsize=17)
    ax.tick_params(axis='x', which='minor', labelsize=16)

    ax.tick_params(axis='y', labelsize=17)

    ax.legend(fontsize=20)

    ax.set_xlabel(label_x, fontsize=20)
    ax.set_ylabel(label_y, fontsize=20)
    ax.set_title(titulo, fontsize=25)

    # Mostrar la gráfica
    plt.show()

In [ ]:
def pintar_resultados(df_ret, test, horizonte, modelo):

    fig, ax = plt.subplots(figsize=(15, 5))
    plt.title(f'{modelo} ({horizonte} horas)', fontsize=30);
    #df_train['temperatura'].plot(ax=ax, label='train')

    df_ret['temperatura'].plot(ax=ax, label='Predicción')
    test.plot(ax=ax, label='Original')
    # datos['temperatura'].tail(25+steps).plot(ax=ax, label='Original')
    ax.legend()
    plt.show()

## - Modelos

### 1. LR

In [ ]:
def LR(X_train, y_train, X_test, y_test, horizonte):
    predicciones = pd.DataFrame()

    model_LR1 = LinearRegression()
    model_LR1.fit(X_train.values, y_train.values)
    y_pred = model_LR1.predict(X_test.values)

    predicciones = pd.DataFrame({"temperatura":y_pred})
    predicciones = predicciones.set_index(X_test.index)

    pintar_resultados(predicciones, y_test['temperatura'], horizonte, 'LR')
    
    r2, mse, mae = obtener_metricas(y_test['temperatura'].values, predicciones)
    resultados = pd.DataFrame({'model':['LR'],
                               'horizonte':[horizonte],
                               'r2':[r2], 
                               'mse':[mse], 
                               'mae':[mae]})
    return resultados, predicciones

### 2. RF

In [ ]:
def RF(X_train, y_train, X_test, y_test, horizonte):
    predicciones = pd.DataFrame()

    model_RF1 = RandomForestRegressor()
    model_RF1.fit(X_train.values, y_train.values)
    y_pred = model_RF1.predict(X_test.values)

    predicciones = pd.DataFrame({"temperatura":y_pred})
    predicciones = predicciones.set_index(X_test.index)

    pintar_resultados(predicciones, y_test['temperatura'], horizonte, 'RF')
    
    r2, mse, mae = obtener_metricas(y_test['temperatura'].values, predicciones)
    resultados = pd.DataFrame({'model':['RF'],
                               'horizonte':[horizonte],
                               'r2':[r2], 
                               'mse':[mse], 
                               'mae':[mae]})
    return resultados, predicciones

### 3. KNN

In [ ]:
def KNN(X_train, y_train, X_test, y_test, horizonte):
    predicciones = pd.DataFrame()

    model_KNN = KNeighborsRegressor(5, weights='distance')
    model_KNN.fit(X_train.values, y_train.values)
    y_pred = model_KNN.predict(X_test.values)

    predicciones=pd.DataFrame({"temperatura":y_pred})
    predicciones = predicciones.set_index(X_test.index)

    pintar_resultados(predicciones, y_test['temperatura'], horizonte, 'KNN')
    
    r2, mse, mae = obtener_metricas(y_test['temperatura'].values, predicciones)
    resultados = pd.DataFrame({'model':['KNN'],
                               'horizonte':[horizonte],
                               'r2':[r2], 
                               'mse':[mse], 
                               'mae':[mae]})
    return resultados, predicciones

### 4. MLP

In [ ]:
def MLP(X_train, y_train, X_test, y_test, horizonte):
    predicciones = pd.DataFrame()

    model_MLP = MLPRegressor(hidden_layer_sizes=(32,32), learning_rate_init=0.01, max_iter=400, random_state=1)
    model_MLP.fit(X_train.values, y_train.values)
    y_pred = model_MLP.predict(X_test.values)

    predicciones=pd.DataFrame({"temperatura":y_pred})
    predicciones = predicciones.set_index(X_test.index)

    pintar_resultados(predicciones, y_test['temperatura'], horizonte, 'MLP')
    
    r2, mse, mae = obtener_metricas(y_test['temperatura'].values, predicciones)
    resultados = pd.DataFrame({'model':['MLP'],
                               'horizonte':[horizonte],
                               'r2':[r2], 
                               'mse':[mse], 
                               'mae':[mae]})
    return resultados, predicciones

### 5. Lasso

In [ ]:
def Lasso_V(X_train, y_train, X_test, y_test, horizonte):
    predicciones = pd.DataFrame()

    model_Lasso = Lasso()
    model_Lasso.fit(X_train.values, y_train.values)
    y_pred = model_Lasso.predict(X_test.values)

    predicciones=pd.DataFrame({"temperatura":y_pred})
    predicciones = predicciones.set_index(X_test.index)

    pintar_resultados(predicciones, y_test['temperatura'], horizonte, 'Lasso')
    
    r2, mse, mae = obtener_metricas(y_test['temperatura'].values, predicciones)
    resultados = pd.DataFrame({'model':['Lasso'],
                               'horizonte':[horizonte],
                               'r2':[r2], 
                               'mse':[mse], 
                               'mae':[mae]})
    return resultados, predicciones

### 6. DT

In [ ]:
def DT(X_train, y_train, X_test, y_test, horizonte):
    predicciones = pd.DataFrame()

    model_DT = DecisionTreeRegressor()
    model_DT.fit(X_train.values, y_train.values)
    y_pred = model_DT.predict(X_test.values)

    predicciones=pd.DataFrame({"temperatura":y_pred})
    predicciones = predicciones.set_index(X_test.index)

    pintar_resultados(predicciones, y_test['temperatura'], horizonte, 'DT')
    
    r2, mse, mae = obtener_metricas(y_test['temperatura'].values, predicciones)
    resultados = pd.DataFrame({'model':['DT'],
                               'horizonte':[horizonte],
                               'r2':[r2], 
                               'mse':[mse], 
                               'mae':[mae]})
    return resultados, predicciones

### 7. SVR

In [ ]:
def SV(X_train, y_train, X_test, y_test, horizonte):
    predicciones = pd.DataFrame()


    model_SVR = SVR()
    model_SVR.fit(X_train.values, y_train.values)
    y_pred = model_SVR.predict(X_test.values)

    predicciones=pd.DataFrame({"temperatura":y_pred})
    predicciones = predicciones.set_index(X_test.index)
    
    pintar_resultados(predicciones, y_test['temperatura'], horizonte, 'SVR')
    
    r2, mse, mae = obtener_metricas(y_test['temperatura'].values, predicciones)
    resultados = pd.DataFrame({'model':['SVR'],
                               'horizonte':[horizonte],
                               'r2':[r2], 
                               'mse':[mse], 
                               'mae':[mae]})
    return resultados, predicciones

### 8. CNN

In [ ]:
def create_model(opt, loss="mse"):
    model = Sequential()
    # opt = tf.optimizers.Adam(learning_rate=0.1)
    # model.add(Dense(32, activation="relu"))
    model.add(Conv1D(64, 2, activation="relu", input_shape=(2, 1)))
    model.add(Flatten())
    model.add(Dense(32, activation="relu"))
    model.add(Dense(1))
    model.compile(loss=loss, optimizer=opt)
    # model.summary()
    return model

def CNN_V(X_train, y_train, X_test, y_test, horizonte, datos_test):
    predicciones = pd.DataFrame()

    opt = "adam"
    model_CNN = create_model(opt)
    model_CNN.fit(X_train, y_train, batch_size=24, epochs=100, verbose=1)
    y_pred = model_CNN.predict(X_test)

    predicciones=pd.DataFrame({"temperatura":y_pred.flatten()})
    predicciones = predicciones.set_index(datos_test.index)

    # pintar_resultados(predicciones, y_test['temperatura'], horizonte, 'CNN')
    
    r2, mse, mae = obtener_metricas(datos_test['temperatura'].values, predicciones)
    resultados = pd.DataFrame({'model':['CNN'],
                               'horizonte':[horizonte],
                               'r2':[r2], 
                               'mse':[mse], 
                               'mae':[mae]})
    
    titulo = f"2do Experimento - CNN ({horizonte} horas)"
    data = [datos_test['temperatura'],predicciones['temperatura']]
    labels = ["Originales", "Predichos"]
    colores = ["cornflowerblue", "limegreen"]#"seagreen","indigo","red","chocolate","orchid","darkgray",gold]
    alto = 3
    ancho = 15
    pintar_grafica(titulo,data, labels, colores, "Fecha","Temperatura (ºC)", alto, ancho)
    
    return resultados, predicciones

### 9. SARIMAX

In [ ]:
def SARIMA_V(X_train, y_train, X_test, y_test, horizonte):
    # fit model
    model_SARIMAX = SARIMAX(y_train, exog=X_train, order=(3, 1, 0), seasonal_order=(1, 0, 1, 24))
    model__SARIMAX_fit = model_SARIMAX.fit(disp=False)
    # make prediction
    predicciones = model__SARIMAX_fit.predict(len(X_train), len(X_train) + len(X_test) -1, exog=X_test.values)
    
    res=pd.DataFrame({'temperatura':predicciones})
    # print(res)
    pintar_resultados(res, y_test['temperatura'], horizonte, 'SARIMA')
    
    r2, mse, mae = obtener_metricas(y_test['temperatura'].values, predicciones)
    resultados = pd.DataFrame({'model':['SARIMA'],
                               'horizonte':[horizonte],
                               'r2':[r2], 
                               'mse':[mse], 
                               'mae':[mae]})
    return resultados, predicciones

### MODELOS

In [ ]:
def ejecutar_modelos(datos_train, datos_test, entreno, resultados):
  X_train = datos_train[["ambiente","nivel"]]
  y_train = datos_train["temperatura"]
  X_test = datos_test[["ambiente","nivel"]]
  y_test = datos_test[["temperatura"]]

  ######################################################

  X_train_CNN = datos_train[["ambiente","nivel"]].to_numpy()
  y_train_CNN = datos_train["temperatura"].to_numpy()
  X_test_CNN = datos_test[["ambiente","nivel"]].to_numpy()
  y_test_CNN = datos_test["temperatura"].to_numpy()

  X_train_CNN = X_train_CNN.reshape(X_train_CNN.shape[0], X_train_CNN.shape[1], 1)
  X_test_CNN = X_test_CNN.reshape(X_test_CNN.shape[0], X_test_CNN.shape[1], 1)

  ##########################################################################################################

  
  res, predicciones_LR = LR(X_train, y_train, X_test, y_test, entreno)
  resultados = pd.concat([resultados, res], axis=0)

  res, predicciones_RF = RF(X_train, y_train, X_test, y_test, entreno)
  resultados = pd.concat([resultados, res], axis=0)

  res, predicciones_KNN = KNN(X_train, y_train, X_test, y_test, entreno)
  resultados = pd.concat([resultados, res], axis=0)

  res, predicciones_MLP = MLP(X_train, y_train, X_test, y_test, entreno)
  resultados = pd.concat([resultados, res], axis=0)

  res, predicciones_Lasso = Lasso_V(X_train, y_train, X_test, y_test, entreno)
  resultados = pd.concat([resultados, res], axis=0)

  res, predicciones_DT = DT(X_train, y_train, X_test, y_test, entreno)
  resultados = pd.concat([resultados, res], axis=0)

  res, predicciones_SVR = SV(X_train, y_train, X_test, y_test, entreno)
  resultados = pd.concat([resultados, res], axis=0)

  res, predicciones_CNN = CNN_V(X_train_CNN, y_train_CNN, X_test_CNN, y_test_CNN, entreno, datos_test)
  resultados = pd.concat([resultados, res], axis=0)

  res, predicciones_SARIMAX = SARIMA_V(X_train, y_train, X_test, y_test, entreno)
  resultados = pd.concat([resultados, res], axis=0)
  
  predicciones_todas = [predicciones_LR, predicciones_RF, predicciones_KNN, 
                  predicciones_MLP, predicciones_Lasso, predicciones_DT,
                  predicciones_SVR, predicciones_CNN, predicciones_SARIMAX]

  # predicciones_todas = [predicciones_SVR, predicciones_SARIMAX]

  return resultados, predicciones_todas

# Resultados

In [ ]:
# DATASET 3
resultados = pd.DataFrame()

predicciones_todas = []

for hor in HORIZONTES_3:
    datos_train, datos_test = separar_train_test(datos, TERCER_CONJUNTO, hor)
    resultados, predicciones = ejecutar_modelos(datos_train, datos_test, hor, resultados)
    predicciones_todas.append(predicciones)

In [ ]:
predicciones_todas

In [ ]:
resultados.reset_index(drop=True, inplace=True)
resultados

In [ ]:
resultados.to_excel('resultados.xlsx', sheet_name='Experimentos_2',index=False)


In [ ]:
df = pd.DataFrame(resultados)
df = df.round(2)
latex_code = df.to_latex(index=False, float_format="%.2f")

In [ ]:
print(latex_code)